In [1]:
import pandas as pd
import numpy as np
import scipy
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from copy import deepcopy
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.interpolate import CubicSpline
from sklearn.metrics import mean_squared_error
import warnings
import itertools
warnings.filterwarnings("ignore")
import arviz as az
import scipy
import matplotlib.patches as patches
import statsmodels.stats.api as sms
import scipy.stats.kde as kde

# Read 16S data

In [2]:
df_16S = pd.read_excel('./16S_relative_abundance.xlsx', index_col=0)
df_16S.columns = [c.replace('/','_slash_').replace(' ','_space_').replace('[','_leftsquarebracket_').replace(']','_rightsquarebracket_').replace('-','_dash_').replace('.','_dot_').replace('(','_leftroundbracket').replace(')','_rightroundbracket_') for c in df_16S.columns]
df_16S.head()

,Abiotrophia_dash_defectiva,Absiella_dash_argi,Acholeplasmatales_dash_bacterium_dash_canine_dash_oral_dash_taxon_dash_316,Acidaminococcus,Acidaminococcus_dash_sp_dot__dash_Marseille_dash_P2764,Acidaminococcus_dash_sp_dot__dash_Marseille_dash_P2828,Acinetobacter,Actinobacillus,Actinobacteria,Actinobacteriota,...,bacterium_dash_mpn_dash_isolate_dash_group_dash_16,bacterium_dash_oral_dash_clone_dash_ASCD05,butyrate_dash_producing_dash_bacterium_dash_SM4_slash_1,gut_dash_metagenome,human_dash_gut_dash_metagenome,metagenome,mouse_dash_gut_dash_metagenome,rumen_dash_bacterium_dash_NK4A78,swine_dash_fecal_dash_bacterium_dash_SD_dash_Xyl5,wallaby_dash_gut_dash_metagenome
SampleID,,,,,,,,,,,,,,,,,,,,,
SRR10022259,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.039097,0.009150,0.009223,0.0,0.000000,0.0,0.000000
SRR10022258,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.041739,0.018898,0.005560,0.0,0.000000,0.0,0.001126
SRR10022120,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.091772,0.073355,0.030477,0.0,0.000000,0.0,0.001085
SRR10022238,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.094728,0.054310,0.017197,0.0,0.006318,0.0,0.000000
SRR10022162,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.094973,0.055338,0.009547,0.0,0.000000,0.0,0.010040


# Add pseudocount for zeros

In [3]:
for sample_id in df_16S.index:
    sample = np.array(df_16S.loc[sample_id])
    minval = np.min(sample[np.nonzero(sample)]) # minimum non-zero value
    sample[sample==0] = minval
    df_16S.loc[sample_id] = sample
df_16S.head()

,Abiotrophia_dash_defectiva,Absiella_dash_argi,Acholeplasmatales_dash_bacterium_dash_canine_dash_oral_dash_taxon_dash_316,Acidaminococcus,Acidaminococcus_dash_sp_dot__dash_Marseille_dash_P2764,Acidaminococcus_dash_sp_dot__dash_Marseille_dash_P2828,Acinetobacter,Actinobacillus,Actinobacteria,Actinobacteriota,...,bacterium_dash_mpn_dash_isolate_dash_group_dash_16,bacterium_dash_oral_dash_clone_dash_ASCD05,butyrate_dash_producing_dash_bacterium_dash_SM4_slash_1,gut_dash_metagenome,human_dash_gut_dash_metagenome,metagenome,mouse_dash_gut_dash_metagenome,rumen_dash_bacterium_dash_NK4A78,swine_dash_fecal_dash_bacterium_dash_SD_dash_Xyl5,wallaby_dash_gut_dash_metagenome
SampleID,,,,,,,,,,,,,,,,,,,,,
SRR10022259,0.000072,0.000072,0.000072,0.000072,0.000072,0.000072,0.000072,0.000072,0.000072,0.000072,...,0.000072,0.000072,0.000072,0.039097,0.009150,0.009223,0.000072,0.000072,0.000072,0.000072
SRR10022258,0.000176,0.000176,0.000176,0.000176,0.000176,0.000176,0.000176,0.000176,0.000176,0.000176,...,0.000176,0.000176,0.000176,0.041739,0.018898,0.005560,0.000176,0.000176,0.000176,0.001126
SRR10022120,0.000186,0.000186,0.000186,0.000186,0.000186,0.000186,0.000186,0.000186,0.000186,0.000186,...,0.000186,0.000186,0.000186,0.091772,0.073355,0.030477,0.000186,0.000186,0.000186,0.001085
SRR10022238,0.000209,0.000209,0.000209,0.000209,0.000209,0.000209,0.000209,0.000209,0.000209,0.000209,...,0.000209,0.000209,0.000209,0.094728,0.054310,0.017197,0.000209,0.006318,0.000209,0.000209
SRR10022162,0.000132,0.000132,0.000132,0.000132,0.000132,0.000132,0.000132,0.000132,0.000132,0.000132,...,0.000132,0.000132,0.000132,0.094973,0.055338,0.009547,0.000132,0.000132,0.000132,0.010040


# Read metadata for maize starch and control

In [4]:
df_meta = pd.read_csv('./metadata.tsv', sep='\t', index_col=0)
df_meta = df_meta.drop('#q2:types')
df_meta.Subject = [s.strip() for s in df_meta.Subject]
df_meta = df_meta[['Treatment','Subject','Dose','week']]
df_meta = df_meta[(df_meta.Treatment=='Maize') | (df_meta.Treatment=='Control')]
df_meta.week = df_meta.week.astype(int)
df_meta = df_meta.rename({'week':'Day'}, axis=1)
df_meta['Diet'] = [1 if g=='Maize' else 0 for g in df_meta['Treatment']]
df_meta = df_meta.drop('Treatment', axis=1)
df_meta.head()

,Subject,Dose,Day,Diet
sample-id,,,,
SRR10022259,F01,baseline,0,0
SRR10022258,F01,10g,1,0
SRR10022120,F01,20g,2,0
SRR10022238,F01,35g,3,0
SRR10022162,F01,50g,4,0


# Select the 20 most abundant taxa

In [5]:
topX = 20
df_16S_T = df_16S.loc[df_meta.index].T
df_16S_T['mean'] = df_16S_T.mean(axis=1)
df_16S_T = df_16S_T.sort_values(by=['mean'],axis=0,ascending=False)
df_16S_T = df_16S_T.drop('mean', axis=1)
df_16S_topX = df_16S_T.iloc[0:topX].T
df_16S_topX.head()

,Bacteroides,Agathobacter,Faecalibacterium,Blautia,Bifidobacterium,gut_dash_metagenome,Prevotella,Faecalibacterium_dash_sp_dot_,Alistipes,Blautia_dash_sp_dot__dash_GD9,Fusicatenibacter,Ruminococcus,Lachnospiraceae,Bifidobacterium_dash_faecale,Anaerostipes,_leftsquarebracket_Ruminococcus_rightsquarebracket__dash_torques_dash_group,Streptococcus,Dorea_dash_longicatena,Roseburia,metagenome
sample-id,,,,,,,,,,,,,,,,,,,,
SRR10022259,0.249412,0.018771,0.047380,0.030200,0.000072,0.039097,0.000072,0.043257,0.030164,0.010886,0.031249,0.028934,0.027524,0.001881,0.013635,0.028030,0.003364,0.020616,0.023220,0.009223
SRR10022258,0.080345,0.015379,0.102939,0.030547,0.012036,0.041739,0.000176,0.007672,0.031357,0.021749,0.019778,0.018335,0.009889,0.000176,0.042231,0.013373,0.074362,0.013197,0.018371,0.005560
SRR10022120,0.097941,0.010634,0.087400,0.023191,0.006635,0.091772,0.000186,0.008309,0.040460,0.003689,0.028276,0.014107,0.019656,0.000186,0.033701,0.008619,0.017362,0.011037,0.004837,0.030477
SRR10022238,0.089289,0.027699,0.084142,0.044017,0.003598,0.094728,0.000209,0.004895,0.025732,0.000209,0.036444,0.002385,0.006151,0.000209,0.029289,0.008452,0.021297,0.010167,0.013933,0.017197
SRR10022162,0.111137,0.033216,0.060506,0.020641,0.011160,0.094973,0.000132,0.005991,0.025974,0.000132,0.014188,0.010238,0.004280,0.000132,0.032854,0.000132,0.025447,0.009152,0.013892,0.009547


In [6]:
list(df_16S_topX.columns)

['Bacteroides',
 'Agathobacter',
 'Faecalibacterium',
 'Blautia',
 'Bifidobacterium',
 'gut_dash_metagenome',
 'Prevotella',
 'Faecalibacterium_dash_sp_dot_',
 'Alistipes',
 'Blautia_dash_sp_dot__dash_GD9',
 'Fusicatenibacter',
 'Ruminococcus',
 'Lachnospiraceae',
 'Bifidobacterium_dash_faecale',
 'Anaerostipes',
 '_leftsquarebracket_Ruminococcus_rightsquarebracket__dash_torques_dash_group',
 'Streptococcus',
 'Dorea_dash_longicatena',
 'Roseburia',
 'metagenome']

# Normalize max value of 16S data to 1

In [7]:
bacterial_taxa = list(df_16S_topX.columns)
df_16S_topX_w_meta = pd.merge(df_meta, df_16S_topX/df_16S_topX.max().max(), left_index=True, right_index=True, how='inner')
df_16S_topX_w_meta.head()

,Subject,Dose,Day,Diet,Bacteroides,Agathobacter,Faecalibacterium,Blautia,Bifidobacterium,gut_dash_metagenome,...,Fusicatenibacter,Ruminococcus,Lachnospiraceae,Bifidobacterium_dash_faecale,Anaerostipes,_leftsquarebracket_Ruminococcus_rightsquarebracket__dash_torques_dash_group,Streptococcus,Dorea_dash_longicatena,Roseburia,metagenome
sample-id,,,,,,,,,,,,,,,,,,,,,
SRR10022259,F01,baseline,0,0,0.758976,0.057121,0.144179,0.091900,0.000220,0.118975,...,0.095092,0.088048,0.083756,0.005723,0.041493,0.085297,0.010236,0.062734,0.070659,0.028065
SRR10022258,F01,10g,1,0,0.244494,0.046800,0.313248,0.092957,0.036626,0.127013,...,0.060186,0.055796,0.030093,0.000535,0.128512,0.040695,0.226288,0.040160,0.055903,0.016921
SRR10022120,F01,20g,2,0,0.298041,0.032361,0.265963,0.070571,0.020190,0.279266,...,0.086044,0.042928,0.059816,0.000566,0.102555,0.026228,0.052834,0.033587,0.014718,0.092743
SRR10022238,F01,35g,3,0,0.271711,0.084289,0.256050,0.133946,0.010950,0.288263,...,0.110900,0.007258,0.018717,0.000637,0.089127,0.025720,0.064808,0.030940,0.042399,0.052330
SRR10022162,F01,50g,4,0,0.338196,0.101078,0.184124,0.062811,0.033960,0.289009,...,0.043176,0.031155,0.013023,0.000401,0.099976,0.000401,0.077436,0.027849,0.042274,0.029051


# Calculate log-derivatives (dlog(B)/dt)

In [8]:
df_deriv = deepcopy(df_16S_topX_w_meta)
for curr_sub in set(df_deriv.Subject):
    curr_df = df_deriv[df_deriv.Subject==curr_sub].sort_values(by='Day')
    for taxon in bacterial_taxa:
        xdata = np.array(curr_df['Day'])
        ydata = np.array(curr_df[taxon])
        cs = CubicSpline(xdata, ydata)
        csd1 = cs.derivative(nu=1)
        ydata_d1 = csd1(xdata)
        df_deriv.loc[df_deriv.Subject==curr_sub, taxon] = ydata_d1
df_deriv.head()

,Subject,Dose,Day,Diet,Bacteroides,Agathobacter,Faecalibacterium,Blautia,Bifidobacterium,gut_dash_metagenome,...,Fusicatenibacter,Ruminococcus,Lachnospiraceae,Bifidobacterium_dash_faecale,Anaerostipes,_leftsquarebracket_Ruminococcus_rightsquarebracket__dash_torques_dash_group,Streptococcus,Dorea_dash_longicatena,Roseburia,metagenome
sample-id,,,,,,,,,,,,,,,,,,,,,
SRR10022259,F01,baseline,0,0,-1.082849,0.029564,0.391247,0.076635,0.085754,-0.195101,...,-0.083392,-0.066387,-0.168460,-0.009926,0.194823,-0.063178,0.665760,-0.035243,0.046220,-0.150409
SRR10022258,F01,10g,1,0,-0.088291,-0.031294,0.003892,-0.042593,-0.001479,0.145661,...,0.004528,-0.010339,0.024583,-0.001514,0.004873,-0.027780,-0.106178,-0.012240,-0.051851,0.080229
SRR10022120,F01,20g,2,0,0.053209,0.021330,-0.041461,0.029750,-0.019929,0.093331,...,0.038135,-0.027618,-0.001690,0.000511,-0.031130,-0.002909,-0.113253,-0.003240,-0.006637,0.023524
SRR10022238,F01,35g,3,0,-0.042897,0.058442,-0.009643,0.046559,0.004167,-0.035233,...,-0.004929,-0.024805,-0.051952,-0.000226,0.001493,-0.005511,0.074751,-0.002462,0.037889,-0.068097
SRR10022162,F01,50g,4,0,0.238840,-0.048948,-0.165485,-0.239268,0.044571,0.076829,...,-0.147023,0.091518,0.069121,-0.000102,0.017423,-0.052529,-0.111945,-0.004126,-0.062248,0.057791


# Construct regression matrix

In [9]:
# Ymat should be samples by bacteria
Ymat = df_deriv[bacterial_taxa].values
Ymat = Ymat.flatten(order='F')
Ymat = StandardScaler().fit_transform(Ymat.reshape(-1,1)).reshape(1,-1)[0] # standardize
Ymat

array([-7.19185011, -0.5861302 ,  0.35369996, ..., -0.00742527,
       -0.06780106,  0.1866441 ])

In [10]:
len(Ymat)

2000

In [11]:
Xmat = np.zeros(shape=(topX*len(df_deriv.index), (topX+2)*topX))
for k in np.arange(topX):
    Xmat[k*len(df_deriv.index):(k+1)*len(df_deriv.index),k*(topX+2)] = 1
    Xmat[k*len(df_deriv.index):(k+1)*len(df_deriv.index),k*(topX+2)+1] = df_deriv.Diet.values
    Xmat[k*len(df_deriv.index):(k+1)*len(df_deriv.index),k*(topX+2)+2:(k+1)*(topX+2)] = df_16S_topX_w_meta[bacterial_taxa].values
Xmat

array([[1.        , 0.        , 0.75897617, ..., 0.        , 0.        ,
        0.        ],
       [1.        , 0.        , 0.2444942 , ..., 0.        , 0.        ,
        0.        ],
       [1.        , 0.        , 0.29804124, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.09230468, 0.03909039,
        0.01626389],
       [0.        , 0.        , 0.        , ..., 0.09130017, 0.08156949,
        0.00660725],
       [0.        , 0.        , 0.        , ..., 0.11636208, 0.0313341 ,
        0.01156015]])

# Write stan program

## write data

In [47]:
json_str = '{\n"N" : %d,\n'%(len(Ymat))

# y variable
json_str += '\"dlogX\" : [%s],\n'%(','.join(list(Ymat.astype(str))))

# x variable
for k1,c1 in enumerate(bacterial_taxa):
    # growth rate
    json_str += '\"growth_rate_%s\" : [%s],\n'%(c1,','.join(list(Xmat[:,k1*(topX+2)].astype(str))))
    # diet response
    json_str += '\"rs_response_%s\" : [%s],\n'%(c1,','.join(list(Xmat[:,k1*(topX+2)+1].astype(str))))
    # bacterial interactions
    for k2,c2 in enumerate(bacterial_taxa):
        v = list(Xmat[:,k1*(topX+2)+2+k2].astype(str))
        json_str += '\"pairwise_interaction_%s_%s\" : [%s]'%(c1,c2,','.join(v))
        if c1 == bacterial_taxa[-1] and c2 == bacterial_taxa[-1]:
            json_str += '\n}'
        else:
            json_str += ',\n'
#print(json_str)

text_file = open("/Users/liaoc/Documents/cmdstan-2.24.1/examples/mice_scfa_3/mice_scfa_3.data.json", "w")
text_file.write("%s" % json_str)
text_file.close()

## write model

In [48]:
# data block
model_str = 'data {\n'
model_str += '\tint<lower=0> N;\n'
model_str += '\tvector[N] dlogX;\n'
for c1 in bacterial_taxa:
    model_str += '\tvector[N] growth_rate_%s;\n'%(c1)
    model_str += '\tvector[N] rs_response_%s;\n'%(c1)
    for c2 in bacterial_taxa:
        model_str += '\tvector[N] pairwise_interaction_%s_%s;\n'%(c1,c2)
model_str += '}\n'

# parameter block
model_str += 'parameters {\n\treal<lower=0,upper=1> sigma;\n'
for c1 in bacterial_taxa:
    model_str += '\treal alpha__%s;\n'%(c1) # growth rate
    model_str += '\treal epsilon__%s;\n'%(c1) # rs response
    for c2 in bacterial_taxa:
        model_str += '\treal beta__%s_%s;\n'%(c1,c2)
model_str += '}\n'       
        
# model block
model_str += 'model {\n\tsigma ~ uniform(0,1);\n'
for c1 in bacterial_taxa:
    model_str += '\talpha__%s ~ normal(0,1);\n'%(c1) # growth rate
    model_str += '\tepsilon__%s ~ normal(0,1);\n'%(c1) # rs response
    for c2 in bacterial_taxa:
        model_str += '\tbeta__%s_%s ~ normal(0,1);\n'%(c1,c2)
model_str += '\tdlogX ~ normal('
for c1 in bacterial_taxa:
    model_str += 'alpha__%s*growth_rate_%s+'%(c1,c1) # growth rate
    model_str += 'epsilon__%s*rs_response_%s+'%(c1,c1) # rs response
    for c2 in bacterial_taxa:
        if c1 == bacterial_taxa[-1] and c2 == bacterial_taxa[-1]:
            model_str += 'beta__%s_%s*pairwise_interaction_%s_%s'%(c1,c2,c1,c2)
        else:
            model_str += 'beta__%s_%s*pairwise_interaction_%s_%s+'%(c1,c2,c1,c2)
model_str += ', sigma);\n}'
#print(model_str)

text_file = open("/Users/liaoc/Documents/cmdstan-2.24.1/examples/mice_scfa_3/mice_scfa_3.stan", "w")
text_file.write("%s" % model_str)
text_file.close()

# Plot stan output

In [12]:
def hpd_grid(sample, alpha=0.05, roundto=2):
    """Calculate highest posterior density (HPD) of array for given alpha. 
    The HPD is the minimum width Bayesian credible interval (BCI). 
    The function works for multimodal distributions, returning more than one mode
    Parameters
    ----------
    
    sample : Numpy array or python list
        An array containing MCMC samples
    alpha : float
        Desired probability of type I error (defaults to 0.05)
    roundto: integer
        Number of digits after the decimal point for the results
    Returns
    ----------
    hpd: array with the lower 
          
    """
    sample = np.asarray(sample)
    sample = sample[~np.isnan(sample)]
    # get upper and lower bounds
    l = np.min(sample)
    u = np.max(sample)
    density = kde.gaussian_kde(sample)
    x = np.linspace(l, u, 2000)
    y = density.evaluate(x)
    #y = density.evaluate(x, l, u) waitting for PR to be accepted
    xy_zipped = zip(x, y/np.sum(y))
    xy = sorted(xy_zipped, key=lambda x: x[1], reverse=True)
    xy_cum_sum = 0
    hdv = []
    for val in xy:
        xy_cum_sum += val[1]
        hdv.append(val[0])
        if xy_cum_sum >= (1-alpha):
            break
    hdv.sort()
    diff = (u-l)/20  # differences of 5%
    hpd = []
    hpd.append(round(min(hdv), roundto))
    for i in range(1, len(hdv)):
        if hdv[i]-hdv[i-1] >= diff:
            hpd.append(round(hdv[i-1], roundto))
            hpd.append(round(hdv[i], roundto))
    hpd.append(round(max(hdv), roundto))
    ite = iter(hpd)
    hpd = list(zip(ite, ite))
    modes = []
    for value in hpd:
        x_hpd = x[(x > value[0]) & (x < value[1])]
        y_hpd = y[(x > value[0]) & (x < value[1])]
        modes.append(round(x_hpd[np.argmax(y_hpd)], roundto))
    return hpd, x, y, modes

In [13]:
fit = az.from_cmdstan(["output_%d.csv"%(i) for i in np.arange(1,4)])

In [14]:
lines = []
    
# basal growth rate
for c in bacterial_taxa:
    var = 'alpha__%s'%(c)
    data = []
    for i in np.arange(0,3):
        data.extend(list(fit.posterior[var][i].values))
    hpd_mu, x_mu, y_mu, modes_mu = hpd_grid(data)
    assert len(hpd_mu) == 1
    (x0, x1) = hpd_mu[0]
    lines.append(['basal_growth_rate', c, x0, x1, (x0+x1)/2, np.mean(data)/np.std(data), x0*x1>0])
    
# inulin response
for c in bacterial_taxa:
    var = 'epsilon__%s'%(c)
    data = []
    for i in np.arange(0,3):
        data.extend(list(fit.posterior[var][i].values))
    hpd_mu, x_mu, y_mu, modes_mu = hpd_grid(data)
    assert len(hpd_mu) == 1
    (x0, x1) = hpd_mu[0]
    lines.append(['rs_response', c, x0, x1, (x0+x1)/2, np.mean(data)/np.std(data), x0*x1>0])

# pairwise interactions
for c1 in bacterial_taxa:
    for c2 in bacterial_taxa:
        var = 'beta__%s_%s'%(c1,c2)
        data = []
        for i in np.arange(0,3):
            data.extend(list(fit.posterior[var][i].values))
        hpd_mu, x_mu, y_mu, modes_mu = hpd_grid(data)
        assert len(hpd_mu) == 1
        (x0, x1) = hpd_mu[0]
        lines.append(['pairwise_interaction', (c1,c2), x0, x1, (x0+x1)/2, np.mean(data)/np.std(data), x0*x1>0])
    
df_stan_output_summary = pd.DataFrame(lines, columns = ['Type','Taxa','Left','Right','Middle','SNR','Significant'])
df_stan_output_summary.head()

,Type,Taxa,Left,Right,Middle,SNR,Significant
0,basal_growth_rate,Bacteroides,-0.53,0.98,0.225,0.585639,False
1,basal_growth_rate,Agathobacter,-1.13,0.37,-0.380,-1.017260,False
2,basal_growth_rate,Faecalibacterium,-0.80,0.70,-0.050,-0.128562,False
3,basal_growth_rate,Blautia,-0.67,0.84,0.085,0.207836,False
4,basal_growth_rate,Bifidobacterium,-0.81,0.68,-0.065,-0.194746,False


In [15]:
df_stan_output_summary.to_excel('bayesian_regression_summary_top20.xlsx')

# Plot

In [17]:
df_stan_output_summary = pd.read_excel('bayesian_regression_summary_top20.xlsx', index_col=0)
df_stan_output_summary = df_stan_output_summary[df_stan_output_summary.Significant==True]
df_stan_output_summary

,Type,Taxa,Left,Right,Middle,SNR,Significant
20,rs_response,Bacteroides,-0.97,-0.11,-0.540,-2.482663,True
40,pairwise_interaction,"('Bacteroides', 'Bacteroides')",-2.12,-0.12,-1.120,-2.226395,True
47,pairwise_interaction,"('Bacteroides', 'Faecalibacterium_dash_sp_dot_')",0.13,3.70,1.915,2.123964,True
60,pairwise_interaction,"('Agathobacter', 'Bacteroides')",0.29,2.26,1.275,2.555938,True
61,pairwise_interaction,"('Agathobacter', 'Agathobacter')",0.38,2.41,1.395,2.719554,True
124,pairwise_interaction,"('Bifidobacterium', 'Bifidobacterium')",-3.35,-0.62,-1.985,-2.849105,True
